In [5]:
import numpy as np
import pandas as pd

def find_key_db_values(db_df:pd.DataFrame):

    #This line replaces anything with "Not Reported" in the column with 0.0
    db_df['Temperature ©'] = db_df['Temperature ©'].replace("Not Reported", 0.0)
    assert (temp_nan := np.count_nonzero(db_df['Temperature ©'].isna())) == 0, f'There are {temp_nan} nan values in the temperature series.'

    #This recalculates the unknown temp values ddG er (kcal/mol)
    #Converts temps to Kelvin
    temps = db_df["Temperature ©"].values.astype(float) + 273.15
    er = db_df['er'].values
    ddG_vals = (temps)*(8.314)*np.log(er)*(0.000239)

    ####Creates Dataframe that averages all reported values of a reactant####
    ddG_df = pd.DataFrame(data=ddG_vals, columns=['ddG er (kcal/mol)'], index=db_df.index)
    react_id_ddG_df = pd.concat([db_df['Reactant ID'],ddG_df], axis=1)
    averaged_ddG_by_react = react_id_ddG_df.groupby('Reactant ID').mean().reset_index()
    average_ddG_react_ser = pd.Series(data=averaged_ddG_by_react['ddG er (kcal/mol)'].values, index=averaged_ddG_by_react['Reactant ID'].values, name='ddG er (kcal/mol)')
    assert np.count_nonzero(average_ddG_react_ser.isna().values) == 0, f'ddG er has nan values: {average_ddG_react_ser.isna()}'

    #This is meant to help isolate and test that certain values aren't present
    isolate_db_df = db_df[['Reactant ID','Product ID','Solvent 1 ID','Solvent 2 ID', 'Oxidant ID', 'Catalyst ID', 'er', 'Olefin Type']]

    #These are assertions to make sure there are not problems in direct comparisons
    assert np.count_nonzero(isolate_db_df['er'].isna().values) == 0, f'ddG er has nan values: {isolate_db_df[isolate_db_df["er"].isna()]}'

    #The following isolate the database such that only tBuOH/Water are used, and the same oxidant
    isolate_db_df = isolate_db_df[isolate_db_df['Solvent 1 ID'] == 'sol1_0']
    isolate_db_df = isolate_db_df[isolate_db_df['Solvent 2 ID'] == 'sol2_0']
    isolate_db_df = isolate_db_df[isolate_db_df['Oxidant ID'] == 'ox_0']

    assert isolate_db_df['Solvent 1 ID'].nunique() == 1, f'There are more than one solvent1 values: {isolate_db_df["Solvent 1 ID"].unique()}'
    assert isolate_db_df['Solvent 2 ID'].nunique() == 1, f'There are more than one solvent2 values: {isolate_db_df["Solvent 2 ID"].unique()}'
    assert isolate_db_df['Oxidant ID'].nunique() == 1, f'There are more than one oxidant ID values: {isolate_db_df["Oxidant ID"].unique()}'

    return average_ddG_react_ser, isolate_db_df


In [6]:
vol_types = ['MaxVol', '3BFSVol']
alk_types = ['Mono', 'Gem', 'Cis', 'Gem', 'Trans', 'TriQ2', 'TriQ3', 'TriQ4', 'Tetra']

db = pd.read_csv('SAD_Database.csv')
average_ddG_react_ser, isolate_db_df = find_key_db_values(db)

for vol in vol_types:
    for alk_type in alk_types:
        desc_df = pd.read_csv(f'7_2_{alk_type}_{vol}_Desc.csv',index_col=0)
        desc_df_no_type = desc_df.loc[:, desc_df.columns != 'Alkene Type']
        alk_type_average_ddG = average_ddG_react_ser[desc_df_no_type.index]

        #Concatenates all values available for the reactant
        ignore_diff_final = pd.concat([desc_df_no_type, alk_type_average_ddG],join='inner',axis=1)

        #Finds the intersection with the unique reactant IDs that follow the above assertions 
        alk_type_react = np.intersect1d(isolate_db_df['Reactant ID'].unique(), desc_df_no_type.index)
        isolate_desc_df = desc_df_no_type.loc[alk_type_react]

        #There are now only 759 unique reactants that are homogeneous for this transformation compared to the original 789 that were available!
        diff_final = pd.concat([isolate_desc_df, alk_type_average_ddG], join='outer', axis=1).dropna()
        print(alk_type, ignore_diff_final.shape)
        ignore_diff_final.to_csv(f'7_4_Ignore_Diff_{alk_type}_{vol}_Desc_Avg.csv')
        diff_final.to_csv(f'7_4_With_Diff_{alk_type}_{vol}_Desc_Avg.csv')


Mono (148, 37)
Gem (148, 44)
Cis (41, 44)
Gem (148, 44)
Trans (250, 44)
TriQ2 (71, 51)
TriQ3 (38, 51)
TriQ4 (67, 51)
Tetra (21, 58)
Mono (148, 37)
Gem (148, 44)
Cis (41, 44)
Gem (148, 44)
Trans (250, 44)
TriQ2 (62, 51)
TriQ3 (37, 51)
TriQ4 (77, 51)
Tetra (21, 58)


#### Within the 1014 Entry Database there are:
4 unique Solvent 1 IDs:
- sol1_0 = tBuOH (973 Entries)
- sol1_1 = MeCN (39 Entries (All Tri))
- sol1_2 = THF (1 entry (Trans))
- sol1_3 = Acetone (1 entry (Trans))

2 unique Solvent 2 IDs:
- sol2_0 = Water (1010 Entries)
- Sol2_1 = Acetone (2 Entries)
- 2 Entries did not have a cosolvent

3 Unique Oxidant IDs:
- ox_0 = K3FeCN6 (1011 Entries)
- ox_1 = NaClO2 (1 Entry (Mono))
- ox_2 = TEMPO (2 entries (Trans))